Importing libraries


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import random
from random import Random
from numpy import log,dot,e,shape
import math
data=pd.read_csv('/content/gdrive/MyDrive/Data_set.csv')
data.head()

Mounted at /content/gdrive


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Dropping id and diagnosis from X and initialising Y as the output variable

In [ ]:
from random import Random
Random(14).shuffle(data.values)
print(data.shape)
# data = data.dropna()
print(data.shape)
y=data.diagnosis
x=data.drop('id',axis=1)
x=x.drop('diagnosis',axis=1)

(569, 32)
(569, 32)


Splitting into training and testing

In [ ]:
x_train=x.iloc[0:382].values
x_test=x.iloc[382:].values
y_train=y.iloc[0:382].values
y_test=y.iloc[382:].values
y_binaryTrain=np.array([1.0 if i == 'M' else 0.0 for i in y_train])
y_binaryTest=np.array([1.0 if i == 'M' else 0.0 for i in y_test])

Feature Engineering (Normalising the data)

In [ ]:
def standardize_data(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    standardized_data = (data - mean) / std
    return standardized_data

x_test = standardize_data(x_test)
x_train = standardize_data(x_train)

mean1 = np.nanmean(x_train)
mean2 = np.nanmean(x_test)
# Replace NaN values with mean
x_train[np.isnan(x_train)] = mean1
x_test[np.isnan(x_test)] = mean2

In [ ]:
n_sample, n_features = x_train.shape
print(n_sample)
print(n_features)
print(len(y_train))

382
30
382


In [ ]:
print(x_train.shape)
n_samples, n_features = x_train.shape
shuffled_indices = np.random.permutation(n_sample)
X_shuffled = x_train[shuffled_indices]
y_shuffled = y_train[shuffled_indices]
print(X_shuffled.shape)
print(y_shuffled.shape)
batch_size = 50
j = 0
X_batch = X_shuffled[j:j+batch_size]
y_batch = y_shuffled[j:j+batch_size]
print(X_batch.shape)
print(y_batch.shape)


(382, 30)
(382, 30)
(382,)
(50, 30)
(50,)


Implementing logistic regression

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function(X, y, w):
    n_samples, n_features = X.shape
    h = sigmoid(X.dot(w))
    J = -1/n_samples * (y.T.dot(np.log(h)) + (1 - y).T.dot(np.log(1 - h)))
    return J

def mini_batch_gradient_descent(X, y, alpha, batch_size, iterations):
    n_samples, n_features = X.shape
    cost_list = np.zeros(iterations)
    w_i = np.zeros(n_samples,dtype=np.float32)
    w=w_i
    for i in range(iterations):
        shuffled_indices = np.random.permutation(n_sample)
        X_shuffled = X[shuffled_indices]
        y_shuffled = y[shuffled_indices]
        
        for j in range(0, n_samples, batch_size):
            X_batch = X_shuffled[j:j+batch_size]
            y_batch = y_shuffled[j:j+batch_size]
            w_batch = w[j:j+batch_size]
            prediction = sigmoid(X_batch.dot(w_batch))
            w_batch = w_batch - alpha*(X_batch.T.dot((prediction - y_batch)))
            cost_list[i] = cost_function(X, y, w)
    
    return w_batch, cost_list

In [ ]:
alpha = 0.01
batch_size = 50
iterations = 1000
w, cost_list = mini_batch_gradient_descent(x_train,y_binaryTrain, alpha, batch_size, iterations)

ValueError: ignored